# [Configuration Settings and Compiling Modes](http://deeplearning.net/software/theano/tutorial/modes.html) 
#환경설정과 컴파일 모드들

Theano의 환경 변수 **우선순위**는 아래와 같다.

* [1] theano.config
* [2] THEANO_FLAGS
* [3] .theanorc

원칙적으로 사용자 code 내에서 [1] theano.config를 수정하지 않고,

[3] [.theanorc](http://deeplearning.net/software/theano/library/config.html#envvar-THEANORC)파일로 환경변수를 치환하거나, 

또 그 변수들을 [2] [THEANO_FLAGS](http://deeplearning.net/software/theano/library/config.html#envvar-THEANO_FLAGS) 를 이용해서 치환할 수 있다.

현재 적용되는 configuration을 theano.config로 print해 볼 수 있다.

In [ ]:
import theano
print theano.config

In [4]:
! python -c 'import theano; print theano.config' |head -60

floatX (('float64', 'float32')) 
    Doc:  Default floating-point precision for python casts
    Value:  float64

warn_float64 (('ignore', 'warn', 'raise', 'pdb')) 
    Doc:  Do an action when a tensor variable with float64 dtype is created. They can't be run on the GPU with the current(old) gpu back-end and are slow with gamer GPUs.
    Value:  ignore

cast_policy (('custom', 'numpy+floatX')) 
    Doc:  Rules for implicit type casting
    Value:  custom

int_division (('int', 'raise', 'floatX')) 
    Doc:  What to do when one computes x / y, where both x and y are of integer types
    Value:  int

device (cpu, gpu*, opencl*, cuda*) 
    Doc:  Default device for computations. If gpu*, change the default to try to move computation to it and to put shared variable of float32 on it. Do not use upper case letters, only lower case even if NVIDIA use capital letters.
    Value:  cpu

gpuarray.init_device (<type 'str'>) 
    Doc:  
             Device to initialize for gpuarray use without mo

### THEANO FLAGS
THEANO-FLAGS 는 콤마(,)로 구분된 key=value 의 목록이다.

THEANO_FLAGS='device=cpu,device=gpu0' 라고 중복해서 써줄 경우에 가장 오른쪽을 인식해서 gpu0를 device로 사용할 것이다.

test.py 라는 파이썬 코드에 configuration을 적용할 경우, 아래와 같이 사용하게 된다.
> THEANP_FLAGS='floatX=float32;device=gpu0;nvcc.fastmath=True' python test.py

|default|values..
--|--|--
config.floatX|'float64'|'float64','float32',...
config.device|'cpu'|'cpu', 'gpu', 'gpu0',... 

### Exercise
연습으로 logistic regression을 생각해 보자..

(1) D[0],D[1],w,b 의 type을 theano.config.floatX에서 지정하는 type으로 치환할 수 있도록함.

(2) 아래 코드가 삽입되었음.

In [ ]:
if any([x.op.__class__.__name__ in ['Gemv', 'CGemv', 'Gemm', 'CGemm'] for x in
        train.maker.fgraph.toposort()]):
    print('Used the cpu')
elif any([x.op.__class__.__name__ in ['GpuGemm', 'GpuGemv'] for x in
          train.maker.fgraph.toposort()]):
    print('Used the gpu')
else:
    print('ERROR, not able to tell if theano used the cpu or the gpu')
    print(train.maker.fgraph.toposort())

In [5]:
! python test_logistic.py

float64
Used the cpu
target values for D
[ 1.  1.  0.  0.  0.  0.  0.  0.  0.  1.]
prediction on D
[1 1 0 0 0 0 0 0 0 1]


In [1]:
! THEANO_FLAGS=device=cpu,floatX=float32 python test_logistic.py

float32
Used the cpu
target values for D
[ 0.  1.  0.  0.  0.  1.  1.  1.  0.  1.]
prediction on D
[0 1 0 0 0 1 1 1 0 1]


In [ ]:
# test_logistic.py

import numpy
import theano
import theano.tensor as T
rng = numpy.random

print theano.config.floatX

N = 400
feats = 784
D = (rng.randn(N, feats).astype(theano.config.floatX),
rng.randint(size=N,low=0, high=2).astype(theano.config.floatX))
training_steps = 10000

# Declare Theano symbolic variables
x = T.matrix("x")
y = T.vector("y")
w = theano.shared(rng.randn(feats).astype(theano.config.floatX), name="w")
b = theano.shared(numpy.asarray(0., dtype=theano.config.floatX), name="b")
x.tag.test_value = D[0]
y.tag.test_value = D[1]

# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w)-b)) # Probability of having a one
prediction = p_1 > 0.5 # The prediction that is done: 0 or 1
xent = -y*T.log(p_1) - (1-y)*T.log(1-p_1) # Cross-entropy
cost = xent.mean() + 0.01*(w**2).sum() # The cost to optimize
gw,gb = T.grad(cost, [w,b])

# Compile expressions to functions
train = theano.function(
            inputs=[x,y],
            outputs=[prediction, xent],
            updates=[(w, w-0.01*gw), (b, b-0.01*gb)],
            name = "train")
predict = theano.function(inputs=[x], outputs=prediction,
            name = "predict")

if any([x.op.__class__.__name__ in ['Gemv', 'CGemv', 'Gemm', 'CGemm'] for x in
        train.maker.fgraph.toposort()]):
    print('Used the cpu')
elif any([x.op.__class__.__name__ in ['GpuGemm', 'GpuGemv'] for x in
          train.maker.fgraph.toposort()]):
    print('Used the gpu')
else:
    print('ERROR, not able to tell if theano used the cpu or the gpu')

for i in range(training_steps):
    pred, err = train(D[0], D[1])

print("target values for D")
print(D[1][:10])

print("prediction on D")
print(predict(D[0])[:10])

Modify and execute this example to run on CPU (the default) with floatX=float32 and time the execution using the command line time python file.py. 

위 예제를 floatX=float32, CPU(default)로 run되도록 수정하여 수행하세요 그리고 time을 적용하세요

### Note
* Apply the Theano flag floatX=float32 (through theano.config.floatX) in your code.
 - floatX=float32 를 code에 적용하세요
 
* Cast inputs before storing them into a shared variable.
 - input을 shared 변수로 저장하기 전에 cast하세요
 
* Circumvent the automatic cast of int32 with float32 to float64:
 - int32의 자동 cast를 피하세요.
 
 - Insert manual cast in your code or use [u]int{8,16}.
 - Insert manual cast around the mean operator (this involves division by length, which is an int64).
 - Note that a new casting mechanism is being developed.

In [26]:
# cast ?
import theano.tensor as T
x = T.iscalar()
x_as_int   = T.cast(x, 'int64')
x_as_float = T.cast(x, 'float32')
print x.dtype
print x_as_int.dtype
print x_as_float.dtype

int32
int64
float32


[solution](http://deeplearning.net/software/theano/_downloads/modes_solution_1.py)

In [6]:
import theano
import theano.tensor as T
x=T.ivector('x')
res1 = x.mean()
res2 = T.cast( x.mean(), 'float32')
out1 = theano.function([x],res1)
out2 = theano.function([x],res2)
print out1([1,2,3]).dtype
print out2([1,2,3]).dtype

float64
float32


### [Mode](http://deeplearning.net/software/theano/library/compile/mode.html#libdoc-compile-mode)

theano.function이 호출될 때마다, input과 output 변수들 사이의 symbolic 관계들은 **optimization & compilation** 된다. 이때 compile이 일어나는 방식은 **compile.mode** 파라미터에 의해서 조절된다. config.mode의 default는 'Mode'=='FAST_RUN'.

mode의 parameter는 2가지로 구성된다. 하나는 optimizer이고 하나는 linker이다.

theano가 사용하는 [optimization](http://deeplearning.net/software/theano/optimizations.html#optimizations) : 그래프를 단순화, 메모리 효율적으로 사용하면서 빠르게 수행되도록 함

[참고]compiler? source code를 object code(기계어)로 바꾸는 프로그램
![compiler](http://www.rtcmagazine.com/files/images/4907/rtc1308_iw_iar_fig01_medium.jpg)
[참고]linker? compiler가 만들어낸 하나 이상의 object files을 가져와 이를 단일 실행 프로그램(.exe), 라이브러리 파일 등으로 병합하는 프로그램이다.[출처:wiki]
![이미지](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b1/Linker.svg/344px-Linker.svg.png)

mode 종류|full constructor|설명
--|--|--
**FAST_RUN** |compile.mode.Mode( linker='cvm', optimizer='fast_run') |모든 가능한 optimization을 적용함('fast_run'). **가능하면 C 구현**('cvm')
**FAST_COMPILE** |compile.mode.Mode( linker='py', optimizer='fast_compile') |graph optimization을 적당히 적용하여 compile함('fast_compile'). **python 으로만 구현**('py')하므로 GPU는 불가능
DebugMode |compile.debugmode.DebugMode()| 모든 optimization을 적용함. 오래걸리지만 문제를 확인하기에 용이
ProfileMode|compile.profilemode.ProfileMode()| (Deprecated-> config.profile 사용권장). 모든 optimization을 적용함. profile을 출력함

* DebugMode와 ProfileMode는 자신만의 linker를 사용 (아래)
* (Deprecated)? 중요도가 떨어져 더 이상 사용되지 않고 앞으로는 사라지게될 기능

### Linker

linker |gc | Raise error by op | Overhead | 정의
--|--|--|--|--
cㅣpy|yes|yes|+++|가능하면 C를 시도하고 python을 사용
cㅣpy_nogc|no|yes|++| gc가 없는 cㅣpy
cvm|yes|yes|++|cㅣpy와 유사하지만 code를 실행하기위한 runtime algorithm은 c를 사용
cmv_nogc|no|yes|+| gc가 없는 cvm
c|no|yes|+| C code만 사용(theano의 op(function)를 처리하다가 error발생할 수 있음)
py|yes|yes|+++| python만 사용
DebugMode|no|yes|very high| Theano가 계산하는 것들을 많이 체크함
ProfileMode|no|no|++++| (Deprecated). 몇몇 부가적인 profiling을 수행함

* gc?(garbage collection 쓰레기모음) 계산 중간 결과들을 모음
* overhead? 어떤 처리를 하기 위해 들어가는 간접적인 처리 시간. 메모리 등

# DebugMode & ProfileMode 사용

만약에 새로운 종류의 expression이나 optimization을 정의하는 경우에 **DebugMode**(오류체크)를 이용하는 것이 유용하다. DebugMode는 self-check를 하고 programing error를 진단하는 것을 돕는다. (굉장히 Overhead가 크기 때문에 development단계에서만 사용해야 함)

### 1. [DebugMode](http://deeplearning.net/software/theano/library/compile/debugmode.html#debugmode)

mode는 아래와 같이 theano.function의 argument로 넣어서 대체할 수도 있다. 

DebugMode는 함수를 compile할 때, 혹은 호출할 때마다 어떤 문제가 감지되면 무엇이 문제인지에 따라 예외를 발생한다.

단점은, 부적합한 input 으로 에러가 나는 경우.. 부적합한 input은 넣어서 수행해 보지 않는 한 문제를 발견할 수 없다.

In [7]:
import theano
import theano.tensor as T
x = T.dvector('x')

f = theano.function([x],x/0)
f([1])

array([ inf])

In [9]:
f = theano.function([x],x/0, mode='DebugMode')
f([1])

InvalidValueError: InvalidValueError
        type(variable) = TensorType(float64, scalar)
        variable       = TensorConstant{inf}
        type(value)    = <type 'numpy.ndarray'>
        dtype(value)   = float64
        shape(value)   = ()
        value          = inf
        min(value)     = inf
        max(value)     = inf
        isfinite       = False
        client_node    = None
        hint           = Graph Input 'TensorConstant{inf}' has invalid value inf
        specific_hint  = none
        context        = ...
  TensorConstant{inf} [@A]

        

### 2. [ProfileMode](http://deeplearning.net/software/theano/library/compile/profilemode.html#profilemode)
(1) ProfileMode Instance 생성하기 

theano.ProfileMode 는 input으로 optimizer와 linker를 받는다. 
(gof.OpWiseCLinker=='clpy')

In [1]:
import theano
import theano.tensor as T
from theano import ProfileMode
profmode = theano.ProfileMode(optimizer='fast_run', linker=theano.gof.OpWiseCLinker())

(2) Graph를 ProfileMode로 Compile하기

In [2]:
v1,v2 = T.vectors(2)
o = v1 * v2
f = theano.function([v1,v2],[o], mode=profmode)
print o.owner.op

Elemwise{mul,no_inplace}


/home/sungmk/anaconda/lib/python2.7/site-packages/theano/compile/profilemode.py:103: UserWarning: DEPRECATION WARNING: The ProfileMode is deprecated. Use the Theano flags/parameter to theano.function 'profile=True' instead of 'mode=ProfileMode'
  "DEPRECATION WARNING: The ProfileMode is deprecated. Use the Theano"


(3) Timing Information 확인

compile한 function을 run한 뒤, profmode.print_summary()를 호출하면, 시간정보를 제공한다.

In [3]:
# run
f([1,2,3],[4,5,6])

[array([  4.,  10.,  18.])]

In [4]:
profmode.print_summary()

ProfileMode is deprecated! Use the new profiler.
 The Theano flags to enable it ise: profile=True
 The Theano flags for the memory profile to it is: profile_memory=True

ProfileMode.print_summary()
---------------------------

Time since import 24.210s
Theano compile time: 0.000s (0.0% since import)
    Optimization time: 0.000s
    Linker time: 0.000s
Theano fct call 0.000s (0.0% since import)
   Theano Op time 0.000s 0.0%(since import) 1.6%(of fct call)
   Theano function overhead in ProfileMode 0.000s 0.0%(since import) 98.4%(of fct call)
1 Theano fct call, 0.000s per call
Rest of the time since import 24.210s 100.0%

Theano fct summary:
<% total fct time> <total time> <time per call> <nb call> <fct name>
   100.0% 0.000s 3.11e-04s 1 <ipython-input-2-d73f1998c5a7>:3

Single Op-wise summary:
<% of local_time spent on this kind of Op> <cumulative %> <self seconds> <cumulative seconds> <time per call> [*] <nb_call> <nb_op> <nb_apply> <Op name>
   100.0%  100.0%  0.000s  0.000s  5.01e-0

[regression.py](https://github.com/Theano/Theano/blob/master/benchmark/regression/regression.py)